# Covid-19 mRNA Vaccine Degradation Prediction

The goal is to design a model that will predict likely degradation rates at each base of an RNA molecule.

In [4]:
# Importing the necessary libraries
import json
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow.keras.layers as L
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Seed for Reproducibility

In [5]:
tf.random.set_seed(2001)
np.random.seed(2001)

## Helper Functions and Useful Variables

In [6]:
# This will tell us the columns we are predicting
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

In [7]:
# Generating random tensors
y_true = tf.random.normal((32, 68, 3))
y_pred = tf.random.normal((32, 68, 3))

In [8]:
def MCRMSE(y_true, y_pred):
    # Calculating the column-wise mean squared error
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    
    # Calculating the mean column-wise root mean squared error
    mcrmse = tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)
    
    return mcrmse

In [9]:
def gru_layer(hidden_dim, dropout):
    # Creating a bidirectional GRU layer
    # hidden_dim: Integer specifying the number of units in the GRU layer
    # dropout: Float specifying the dropout rate to apply to the GRU layer
    # return_sequences=True: Returns the full sequence of outputs instead of just the last output
    # kernel_initializer='orthogonal': Initializes the weights of the GRU layer using an orthogonal initializer
    gru = L.Bidirectional(L.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))
    
    return gru

In [10]:
def build_model(embed_size, seq_len=107, pred_len=68, dropout=0.5, sp_dropout=0.2, embed_dim=200, hidden_dim=256, n_layers=3):
    # inputs: Input layer with shape (seq_len, 3)
    inputs = L.Input(shape=(seq_len, 3))
    
    # embed: Embedding layer to convert input into fixed-size dense vectors
    embed = L.Embedding(input_dim=embed_size, output_dim=embed_dim)(inputs)
    
    # reshaped: Reshaping the embedded input to prepare for dropout
    reshaped = tf.reshape(embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
    
    # hidden: Applying spatial dropout to the reshaped input
    hidden = L.SpatialDropout1D(sp_dropout)(reshaped)
    
    # Looping through the specified number of layers
    for x in range(n_layers):
        # Applying GRU layer with specified hidden_dim and dropout to the hidden state
        hidden = gru_layer(hidden_dim, dropout)(hidden)
    
    # truncated: Selecting the relevant predictions from the hidden state
    truncated = hidden[:, :pred_len]
    
    # out: Dense layer with 5 units and linear activation for final predictions
    out = L.Dense(5, activation='linear')(truncated)
    
    # Creating the Keras model with inputs and outputs
    model = tf.keras.Model(inputs=inputs, outputs=out)
    
    # Compiling the model with Adam optimizer and MCRMSE loss function
    model.compile(tf.optimizers.Adam(), loss=MCRMSE)
    
    return model

In [11]:
def pandas_list_to_array(df):
    """
    Input: dataframe of shape (x, y), containing list of length l
    Return: np.array of shape (x, l, y)
    """
    
    return np.transpose(
        np.array(df.values.tolist()),
        (0, 2, 1)
    )

In [12]:
def preprocess_inputs(df, token2int, cols=['sequence', 'structure', 'predicted_loop_type']):
    # Applying a lambda function to convert each token in the specified columns to its corresponding integer value using token2int mapping
    mapped = df[cols].applymap(lambda seq: [token2int[x] for x in seq])
    
    # Converting the mapped DataFrame to a NumPy array
    preprocessed = pandas_list_to_array(mapped)
    
    return preprocessed

## Data Preprocessing

In [13]:
train = pd.read_json('train.json', lines=True)
test = pd.read_json('test.json', lines=True)

In [14]:
train = train.query("signal_to_noise >= 1")

In [15]:
# We will use this dictionary to map each character to an integer
# so that it can be used as an input in keras
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

train_inputs = preprocess_inputs(train, token2int)
train_labels = pandas_list_to_array(train[pred_cols])

In [16]:
x_train, x_val, y_train, y_val = train_test_split(
    train_inputs, train_labels, test_size=.1, random_state=34, stratify=train.SN_filter)

In [17]:
# Filtering the test DataFrame to create a public DataFrame with seq_length == 107
public_df = test.query("seq_length == 107")

# Filtering the test DataFrame to create a private DataFrame with seq_length == 130
private_df = test.query("seq_length == 130")

# Preprocessing the inputs for the public DataFrame using the preprocess_inputs function
public_inputs = preprocess_inputs(public_df, token2int)

# Preprocessing the inputs for the private DataFrame using the preprocess_inputs function
private_inputs = preprocess_inputs(private_df, token2int)

## Modelling

We will train a bi-directional GRU model with 3 layers + dropout

In [ ]:
# Creating a model using the build_model function and passing the length of the token2int mapping as embed_size
model = build_model(embed_size=len(token2int))

# Printing a summary of the model's architecture
model.summary()

In [ ]:
# Training the model using the fit method
history = model.fit(
    x_train, y_train,  # Training data and labels
    validation_data=(x_val, y_val),  # Validation data and labels
    batch_size=64,  # Number of samples per gradient update
    epochs=75,  # Number of epochs to train the model
    verbose=2,  # Verbosity mode: 0 = silent, 1 = progress bar, 2 = one line per epoch
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(patience=5),  # Callback to reduce learning rate on plateau
        tf.keras.callbacks.ModelCheckpoint('model.h5')  # Callback to save the model checkpoint
    ]
)

## Training History Evaluation

In [ ]:
fig = px.line(
    history.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'MCRMSE'}, 
    title='Training History')
fig.show()

## Model Loading + Prediction

In [18]:
# Creating a model for public sequences with seq_len and pred_len set to 107
model_public = build_model(seq_len=107, pred_len=107, embed_size=len(token2int))

# Creating a model for private sequences with seq_len and pred_len set to 130
model_private = build_model(seq_len=130, pred_len=130, embed_size=len(token2int))

# Loading the weights of the trained model from 'model.h5' into both models
model_public.load_weights('model.h5')
model_private.load_weights('model.h5')

In [19]:
public_preds = model_public.predict(public_inputs)
private_preds = model_private.predict(private_inputs)

## Post-processing

In [ ]:
preds_ls = []

# Iterating over the public_df and public_preds, and private_df and private_preds simultaneously
for df, preds in [(public_df, public_preds), (private_df, private_preds)]:
    # Iterating over each id and its corresponding predictions
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        # Creating a DataFrame from the single prediction and adding id_seqpos column
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        # Appending the single DataFrame to the preds_ls list
        preds_ls.append(single_df)

# Concatenating the list of DataFrames into a single DataFrame
preds_df = pd.concat(preds_ls)

# Displaying the first few rows of the concatenated DataFrame
preds_df.head()